In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, make_scorer
import plotly.figure_factory as ff
import tensorflow as tf
from tensorflow.python.keras.layers import Dense, Dropout, LSTM
from tensorflow.python.keras import Sequential
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import OrdinalEncoder
from pandas import DataFrame
from pandas import concat
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_curve, precision_score, recall_score, f1_score
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_validate, GridSearchCV
import time
import seaborn as sns
import sklearn

Using TensorFlow backend.


In [2]:
house_data = pd.read_csv("../preprocessed_data/oneHot_data.csv")

In [3]:
house_data.head()

,hour_sin,hour_cos,month_sin,month_cos,day_sin,day_cos,Month,DayOfWeek,Day,HourofDay,...,T_stp_heat,Humidity,auxHeat1,fan,Thermostat_Temperature,Thermostat_Motion,Remote_Sensor_1_Temperature,Remote_Sensor_1_Motion,T_out,RH_out
0,0.0,1.0,0.0,1.0,0.755303,0.484974,1,1,1,0,...,0.663321,0.177458,0,11,0.330465,0.0,67.0,0.0,0.009112,1.808865
1,0.0,1.0,0.0,1.0,0.755303,0.484974,1,1,1,0,...,0.663321,0.177458,2,9,0.330465,0.0,67.0,0.0,0.009112,1.808865
2,0.0,1.0,0.0,1.0,0.755303,0.484974,1,1,1,0,...,0.663321,0.177458,20,20,0.330465,0.0,67.0,0.0,0.009112,1.808865
3,0.0,1.0,0.0,1.0,0.755303,0.484974,1,1,1,0,...,0.663321,0.177458,9,12,0.330465,0.0,67.0,0.0,0.009112,1.808865
4,0.0,1.0,0.0,1.0,0.755303,0.484974,1,1,1,0,...,0.663321,0.177458,0,8,0.330465,0.0,67.0,0.0,0.009112,1.808865


In [4]:
house_data.columns

Index(['hour_sin', 'hour_cos', 'month_sin', 'month_cos', 'day_sin', 'day_cos',
       'Month', 'DayOfWeek', 'Day', 'HourofDay', 'HvacMode_heat', 'Event_Hold',
       'Event_None', 'Schedule_Away', 'Schedule_Home', 'Schedule_Sleep',
       'T_ctrl', 'T_stp_heat', 'Humidity', 'auxHeat1', 'fan',
       'Thermostat_Temperature', 'Thermostat_Motion',
       'Remote_Sensor_1_Temperature', 'Remote_Sensor_1_Motion', 'T_out',
       'RH_out'],
      dtype='object')

In [5]:
working_df = house_data[['T_ctrl', 'T_stp_heat', 'auxHeat1', 'T_out', 'RH_out']]
cols = ['T_ctrl', 'T_stp_heat', 'T_out', 'RH_out', 'auxHeat1']
working_df = working_df[cols]

In [6]:
working_df.head()

,T_ctrl,T_stp_heat,T_out,RH_out,auxHeat1
0,0.311691,0.663321,0.009112,1.808865,0
1,0.311691,0.663321,0.009112,1.808865,2
2,0.311691,0.663321,0.009112,1.808865,20
3,0.311691,0.663321,0.009112,1.808865,9
4,0.311691,0.663321,0.009112,1.808865,0


In [7]:
def series_to_supervised(data, n_in = 1, n_out = 1, dropnan = True):
    
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

### Model House

In [8]:
reframed = series_to_supervised(working_df, 6, 1)

In [9]:
reframed

,var1(t-6),var2(t-6),var3(t-6),var4(t-6),var5(t-6),var1(t-5),var2(t-5),var3(t-5),var4(t-5),var5(t-5),...,var1(t-1),var2(t-1),var3(t-1),var4(t-1),var5(t-1),var1(t),var2(t),var3(t),var4(t),var5(t)
6,0.311691,0.663321,0.009112,1.808865,0.0,0.311691,0.663321,0.009112,1.808865,2.0,...,0.311691,0.663321,0.009112,1.808865,0.0,0.311691,0.663321,0.075007,1.877919,0
7,0.311691,0.663321,0.009112,1.808865,2.0,0.311691,0.663321,0.009112,1.808865,20.0,...,0.311691,0.663321,0.075007,1.877919,0.0,0.311691,0.663321,0.075007,1.877919,0
8,0.311691,0.663321,0.009112,1.808865,20.0,0.311691,0.663321,0.009112,1.808865,9.0,...,0.311691,0.663321,0.075007,1.877919,0.0,0.311691,0.663321,0.075007,1.877919,14
9,0.311691,0.663321,0.009112,1.808865,9.0,0.311691,0.663321,0.009112,1.808865,0.0,...,0.311691,0.663321,0.075007,1.877919,14.0,0.311691,0.663321,0.075007,1.877919,19
10,0.311691,0.663321,0.009112,1.808865,0.0,0.311691,0.663321,0.009112,1.808865,0.0,...,0.311691,0.663321,0.075007,1.877919,19.0,0.311691,0.663321,0.075007,1.877919,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58832,0.311691,0.663321,-0.649837,-0.055597,10.0,0.311691,0.663321,-0.649837,-0.055597,20.0,...,0.311691,0.663321,-0.649837,-0.124651,0.0,0.311691,0.663321,-0.649837,-0.124651,0
58833,0.311691,0.663321,-0.649837,-0.055597,20.0,0.311691,0.663321,-0.649837,-0.055597,20.0,...,0.311691,0.663321,-0.649837,-0.124651,0.0,0.311691,0.663321,-0.649837,-0.124651,0
58834,0.311691,0.663321,-0.649837,-0.055597,20.0,0.311691,0.663321,-0.649837,-0.055597,9.0,...,0.311691,0.663321,-0.649837,-0.124651,0.0,0.311691,0.663321,-0.649837,-0.124651,0
58835,0.311691,0.663321,-0.649837,-0.055597,9.0,0.311691,0.663321,-0.649837,-0.055597,0.0,...,0.311691,0.663321,-0.649837,-0.124651,0.0,0.311691,0.663321,-0.649837,-0.124651,0


In [10]:
y_auxHeat = reframed['var5(t)']
#y_auxHeat = to_categorical(y_auxHeat)
X = reframed.drop(labels = ["var5(t)"], axis = 1)

In [11]:
# X_train, X_test, y_train, y_test = train_test_split(X, y_auxHeat, test_size = 0.2, shuffle = True)

In [12]:
# X_train = X_train.values
# X_test = X_test.values

In [13]:
# print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

In [14]:
def build_classifier(optimizer):

    classifier = Sequential()

    # Adding the input layer and the first hidden layer
    classifier.add(Dense(units = 24, kernel_initializer = 'uniform', activation = 'relu', input_dim = 34))

    # Adding the second hidden layer
    classifier.add(Dense(units = 24, kernel_initializer = 'uniform', activation = 'relu'))

    # Adding the output layer
    classifier.add(Dense(units = 21, kernel_initializer = 'uniform', activation = 'softmax'))

    # Compiling the ANN
    classifier.compile(optimizer = optimizer, loss = 'categorical_crossentropy', metrics = ['accuracy'])
    
    return classifier

In [15]:
classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, nb_epoch = 20)

In [19]:
scoring = {'acc': 'accuracy'}

parameters = {'batch_size' : [10, 64],
              'epochs' : [10, 50],
              'optimizer' : ['adam', 'rmsprop']}

In [20]:
grid_search = GridSearchCV(estimator = classifier,
                           param_grid = parameters,
                           scoring = scoring,
                           refit = False,
                           cv = 10)

In [21]:
grid_search.fit(X, y_auxHeat)

Epoch 1/10
5295/5295 [==============================] - 14s 3ms/step - loss: 0.7399 - accuracy: 0.8528
Epoch 2/10
5295/5295 [==============================] - 14s 3ms/step - loss: 0.6397 - accuracy: 0.8684
Epoch 3/10
5295/5295 [==============================] - 14s 3ms/step - loss: 0.6211 - accuracy: 0.8697
Epoch 4/10
5295/5295 [==============================] - 14s 3ms/step - loss: 0.6124 - accuracy: 0.8705
Epoch 5/10
5295/5295 [==============================] - 14s 3ms/step - loss: 0.6070 - accuracy: 0.8712
Epoch 6/10
5295/5295 [==============================] - 14s 3ms/step - loss: 0.6024 - accuracy: 0.8715
Epoch 7/10
5295/5295 [==============================] - 14s 3ms/step - loss: 0.5985 - accuracy: 0.8719
Epoch 8/10
5295/5295 [==============================] - 14s 3ms/step - loss: 0.5957 - accuracy: 0.8720
Epoch 9/10
5295/5295 [==============================] - 14s 3ms/step - loss: 0.5929 - accuracy: 0.8722
Epoch 10/10
5295/5295 [==============================] - 14s 3ms/step - l

5295/5295 [==============================] - 17s 3ms/step - loss: 0.8326 - accuracy: 0.8422
Epoch 10/10
5295/5295 [==============================] - 17s 3ms/step - loss: 0.8405 - accuracy: 0.8424
Epoch 1/10
5295/5295 [==============================] - 17s 3ms/step - loss: 0.8492 - accuracy: 0.8288
Epoch 2/10
5295/5295 [==============================] - 17s 3ms/step - loss: 0.7590 - accuracy: 0.8453
Epoch 3/10
5295/5295 [==============================] - 17s 3ms/step - loss: 0.7681 - accuracy: 0.8447
Epoch 4/10
5295/5295 [==============================] - 17s 3ms/step - loss: 0.7847 - accuracy: 0.8428
Epoch 5/10
5295/5295 [==============================] - 17s 3ms/step - loss: 0.8002 - accuracy: 0.8428
Epoch 6/10
5295/5295 [==============================] - 17s 3ms/step - loss: 0.8078 - accuracy: 0.8422
Epoch 7/10
5295/5295 [==============================] - 17s 3ms/step - loss: 0.8097 - accuracy: 0.8408
Epoch 8/10
5295/5295 [==============================] - 17s 3ms/step - loss: 0.8186

5295/5295 [==============================] - 13s 2ms/step - loss: 0.5748 - accuracy: 0.8747
Epoch 18/50
5295/5295 [==============================] - 13s 2ms/step - loss: 0.5744 - accuracy: 0.8748
Epoch 19/50
5295/5295 [==============================] - 13s 2ms/step - loss: 0.5729 - accuracy: 0.8751
Epoch 20/50
5295/5295 [==============================] - 13s 2ms/step - loss: 0.5722 - accuracy: 0.8748
Epoch 21/50
5295/5295 [==============================] - 13s 2ms/step - loss: 0.5723 - accuracy: 0.8753
Epoch 22/50
5295/5295 [==============================] - 13s 2ms/step - loss: 0.5710 - accuracy: 0.8753
Epoch 23/50
5295/5295 [==============================] - 13s 2ms/step - loss: 0.5698 - accuracy: 0.8752
Epoch 24/50
5295/5295 [==============================] - 13s 2ms/step - loss: 0.5693 - accuracy: 0.8755
Epoch 25/50
5295/5295 [==============================] - 13s 2ms/step - loss: 0.5690 - accuracy: 0.8754
Epoch 26/50
5295/5295 [==============================] - 13s 3ms/step - loss

Epoch 24/50
5295/5295 [==============================] - 13s 3ms/step - loss: 0.6557 - accuracy: 0.8555
Epoch 25/50
5295/5295 [==============================] - 14s 3ms/step - loss: 0.6555 - accuracy: 0.8554
Epoch 26/50
5295/5295 [==============================] - 13s 3ms/step - loss: 0.6539 - accuracy: 0.8555
Epoch 27/50
5295/5295 [==============================] - 13s 3ms/step - loss: 0.6542 - accuracy: 0.8553
Epoch 28/50
5295/5295 [==============================] - 14s 3ms/step - loss: 0.6530 - accuracy: 0.8555
Epoch 29/50
5295/5295 [==============================] - 14s 3ms/step - loss: 0.6528 - accuracy: 0.8557
Epoch 30/50
5295/5295 [==============================] - 14s 3ms/step - loss: 0.6518 - accuracy: 0.8554
Epoch 31/50
5295/5295 [==============================] - 14s 3ms/step - loss: 0.6523 - accuracy: 0.8556
Epoch 32/50
5295/5295 [==============================] - 14s 3ms/step - loss: 0.6513 - accuracy: 0.8558
Epoch 33/50
5295/5295 [==============================] - 14s 3ms

5295/5295 [==============================] - 14s 3ms/step - loss: 0.5804 - accuracy: 0.8719
Epoch 32/50
5295/5295 [==============================] - 14s 3ms/step - loss: 0.5808 - accuracy: 0.8721
Epoch 33/50
5295/5295 [==============================] - 14s 3ms/step - loss: 0.5796 - accuracy: 0.8725
Epoch 34/50
5295/5295 [==============================] - 14s 3ms/step - loss: 0.5792 - accuracy: 0.8721
Epoch 35/50
5295/5295 [==============================] - 14s 3ms/step - loss: 0.5797 - accuracy: 0.8723
Epoch 36/50
5295/5295 [==============================] - 14s 3ms/step - loss: 0.5783 - accuracy: 0.8722
Epoch 37/50
5295/5295 [==============================] - 14s 3ms/step - loss: 0.5787 - accuracy: 0.8720
Epoch 38/50
5295/5295 [==============================] - 14s 3ms/step - loss: 0.5779 - accuracy: 0.8724
Epoch 39/50
5295/5295 [==============================] - 13s 3ms/step - loss: 0.5781 - accuracy: 0.8721
Epoch 40/50
5295/5295 [==============================] - 14s 3ms/step - loss

...

828/828 [==============================] - 2s 3ms/step - loss: 0.5834 - accuracy: 0.8720
Epoch 31/50
828/828 [==============================] - 2s 3ms/step - loss: 0.5827 - accuracy: 0.8722
Epoch 32/50
828/828 [==============================] - 2s 3ms/step - loss: 0.5828 - accuracy: 0.8719
Epoch 33/50
828/828 [==============================] - 2s 3ms/step - loss: 0.5817 - accuracy: 0.8723
Epoch 34/50
828/828 [==============================] - 2s 3ms/step - loss: 0.5813 - accuracy: 0.8721
Epoch 35/50
828/828 [==============================] - 2s 3ms/step - loss: 0.5808 - accuracy: 0.8721
Epoch 36/50
828/828 [==============================] - 2s 3ms/step - loss: 0.5803 - accuracy: 0.8722
Epoch 37/50
828/828 [==============================] - 2s 3ms/step - loss: 0.5807 - accuracy: 0.8722
Epoch 38/50
828/828 [==============================] - 2s 3ms/step - loss: 0.5802 - accuracy: 0.8720
Epoch 39/50
828/828 [==============================] - 2s 3ms/step - loss: 0.5796 - accuracy: 0.8724
Ep

828/828 [==============================] - 3s 3ms/step - loss: 0.5895 - accuracy: 0.8749
Epoch 43/50
828/828 [==============================] - 3s 3ms/step - loss: 0.5885 - accuracy: 0.8755
Epoch 44/50
828/828 [==============================] - 3s 3ms/step - loss: 0.5895 - accuracy: 0.8756
Epoch 45/50
828/828 [==============================] - 3s 3ms/step - loss: 0.5906 - accuracy: 0.8753
Epoch 46/50
828/828 [==============================] - 3s 3ms/step - loss: 0.5897 - accuracy: 0.8754
Epoch 47/50
828/828 [==============================] - 3s 3ms/step - loss: 0.5906 - accuracy: 0.8751
Epoch 48/50
828/828 [==============================] - 3s 3ms/step - loss: 0.5930 - accuracy: 0.8749
Epoch 49/50
828/828 [==============================] - 3s 3ms/step - loss: 0.5949 - accuracy: 0.8749
Epoch 50/50
828/828 [==============================] - 3s 3ms/step - loss: 0.5928 - accuracy: 0.8750
Epoch 1/50
828/828 [==============================] - 3s 3ms/step - loss: 0.8722 - accuracy: 0.8310
Epo

828/828 [==============================] - 3s 3ms/step - loss: 0.7093 - accuracy: 0.8501
Epoch 5/50
828/828 [==============================] - 3s 3ms/step - loss: 0.7000 - accuracy: 0.8513
Epoch 6/50
828/828 [==============================] - 3s 3ms/step - loss: 0.6936 - accuracy: 0.8520
Epoch 7/50
828/828 [==============================] - 3s 3ms/step - loss: 0.6893 - accuracy: 0.8531
Epoch 8/50
828/828 [==============================] - 3s 3ms/step - loss: 0.6868 - accuracy: 0.8527
Epoch 9/50
828/828 [==============================] - 3s 3ms/step - loss: 0.6832 - accuracy: 0.8534
Epoch 10/50
828/828 [==============================] - 3s 3ms/step - loss: 0.6807 - accuracy: 0.8537
Epoch 11/50
828/828 [==============================] - 3s 3ms/step - loss: 0.6791 - accuracy: 0.8539
Epoch 12/50
828/828 [==============================] - 3s 3ms/step - loss: 0.6758 - accuracy: 0.8542
Epoch 13/50
828/828 [==============================] - 3s 3ms/step - loss: 0.6751 - accuracy: 0.8540
Epoch 1

Epoch 16/50
828/828 [==============================] - 3s 3ms/step - loss: 0.6005 - accuracy: 0.8715
Epoch 17/50
828/828 [==============================] - 3s 3ms/step - loss: 0.6017 - accuracy: 0.8712
Epoch 18/50
828/828 [==============================] - 3s 3ms/step - loss: 0.6002 - accuracy: 0.8715
Epoch 19/50
828/828 [==============================] - 3s 3ms/step - loss: 0.6003 - accuracy: 0.8717
Epoch 20/50
828/828 [==============================] - 3s 3ms/step - loss: 0.5994 - accuracy: 0.8714
Epoch 21/50
828/828 [==============================] - 3s 3ms/step - loss: 0.5993 - accuracy: 0.8718
Epoch 22/50
828/828 [==============================] - 3s 3ms/step - loss: 0.5990 - accuracy: 0.8717
Epoch 23/50
828/828 [==============================] - 3s 3ms/step - loss: 0.5990 - accuracy: 0.8716
Epoch 24/50
828/828 [==============================] - 3s 3ms/step - loss: 0.6002 - accuracy: 0.8718
Epoch 25/50
828/828 [==============================] - 3s 3ms/step - loss: 0.6000 - accurac

GridSearchCV(cv=10, error_score=nan,
             estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x000002C17A253208>,
             iid='deprecated', n_jobs=None,
             param_grid={'batch_size': [10, 64], 'epochs': [10, 50],
                         'optimizer': ['adam', 'rmsprop']},
             pre_dispatch='2*n_jobs', refit=False, return_train_score=False,
             scoring={'acc': 'accuracy'}, verbose=0)

In [23]:
grid_search.cv_results_

{'mean_fit_time': array([151.90045807, 176.7957392 , 700.22740772, 832.07307525,
         22.39152536,  26.92694736, 108.42340109, 131.38755593]),
 'std_fit_time': array([12.8638599 ,  7.57193344, 22.42274982, 16.54500364,  0.20245908,
         0.27397983,  1.2614378 ,  2.79428255]),
 'mean_score_time': array([0.43891954, 0.40826263, 0.42205403, 0.4240098 , 0.12088656,
        0.11949196, 0.12133467, 0.11818781]),
 'std_score_time': array([0.07536725, 0.01499165, 0.01514787, 0.01057658, 0.00442487,
        0.00257554, 0.01468665, 0.0083219 ]),
 'param_batch_size': masked_array(data=[10, 10, 10, 10, 64, 64, 64, 64],
              mask=[False, False, False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_epochs': masked_array(data=[10, 10, 50, 50, 10, 10, 50, 50],
              mask=[False, False, False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_optimizer': masked_array(data=['adam', 'rm

In [24]:
grid_search.get_params

<bound method BaseEstimator.get_params of GridSearchCV(cv=10, error_score=nan,
             estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x000002C17A253208>,
             iid='deprecated', n_jobs=None,
             param_grid={'batch_size': [10, 64], 'epochs': [10, 50],
                         'optimizer': ['adam', 'rmsprop']},
             pre_dispatch='2*n_jobs', refit=False, return_train_score=False,
             scoring={'acc': 'accuracy'}, verbose=0)>

In [30]:
grid_search_results = pd.DataFrame(grid_search.cv_results_, columns = list(grid_search.cv_results_.keys()))

In [31]:
grid_search_results

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_batch_size,param_epochs,param_optimizer,params,split0_test_acc,split1_test_acc,...,split3_test_acc,split4_test_acc,split5_test_acc,split6_test_acc,split7_test_acc,split8_test_acc,split9_test_acc,mean_test_acc,std_test_acc,rank_test_acc
0,151.900458,12.863860,0.438920,0.075367,10,10,adam,"{'batch_size': 10, 'epochs': 10, 'optimizer': ...",0.808124,0.781574,...,0.881863,0.928268,0.973823,0.982662,0.855006,0.821690,0.826789,0.864885,0.070265,3
1,176.795739,7.571933,0.408263,0.014992,10,10,rmsprop,"{'batch_size': 10, 'epochs': 10, 'optimizer': ...",0.759687,0.736529,...,0.879823,0.921299,0.971783,0.977902,0.843617,0.806391,0.820500,0.849027,0.081834,7
2,700.227408,22.422750,0.422054,0.015148,10,50,adam,"{'batch_size': 10, 'epochs': 50, 'optimizer': ...",0.808973,0.787183,...,0.879993,0.926568,0.972803,0.982662,0.853476,0.822030,0.827809,0.865463,0.068703,2
3,832.073075,16.545004,0.424010,0.010577,10,50,rmsprop,"{'batch_size': 10, 'epochs': 50, 'optimizer': ...",0.536540,0.622811,...,0.826789,0.922489,0.962604,0.978412,0.788543,0.813021,0.641169,0.770193,0.150902,8
4,22.391525,0.202459,0.120887,0.004425,64,10,adam,"{'batch_size': 64, 'epochs': 10, 'optimizer': ...",0.803535,0.780894,...,0.882713,0.928438,0.973483,0.982832,0.850246,0.819650,0.827979,0.863831,0.070936,5
5,26.926947,0.273980,0.119492,0.002576,64,10,rmsprop,"{'batch_size': 64, 'epochs': 10, 'optimizer': ...",0.804215,0.780384,...,0.880843,0.926908,0.973483,0.983002,0.850586,0.818290,0.828319,0.863100,0.071228,6
6,108.423401,1.261438,0.121335,0.014687,64,50,adam,"{'batch_size': 64, 'epochs': 50, 'optimizer': ...",0.809653,0.785144,...,0.882203,0.927418,0.973143,0.983172,0.856026,0.822030,0.831039,0.866143,0.069099,1
7,131.387556,2.794283,0.118188,0.008322,64,50,rmsprop,"{'batch_size': 64, 'epochs': 50, 'optimizer': ...",0.806594,0.775115,...,0.881863,0.928778,0.973483,0.983342,0.852116,0.820330,0.827129,0.863984,0.071178,4


In [32]:
grid_search_results.to_csv("./gridSearchResults.csv")

In [29]:
list(grid_search.cv_results_.keys())

['mean_fit_time',
 'std_fit_time',
 'mean_score_time',
 'std_score_time',
 'param_batch_size',
 'param_epochs',
 'param_optimizer',
 'params',
 'split0_test_acc',
 'split1_test_acc',
 'split2_test_acc',
 'split3_test_acc',
 'split4_test_acc',
 'split5_test_acc',
 'split6_test_acc',
 'split7_test_acc',
 'split8_test_acc',
 'split9_test_acc',
 'mean_test_acc',
 'std_test_acc',
 'rank_test_acc']

In [ ]:
start = time.time()
scores = cross_validate(estimator = classifier, X = X, y = y_auxHeat, scoring = scoring, cv = 10)
print(scores.keys())
print(scores['test_acc'])

In [ ]:
sns.boxplot(scores['test_acc'])

In [ ]:
print(np.mean(scores['test_f1_weighted']))